Importa as bibliotecas necessárias

In [1]:
import boto3
import pandas as pd
import io
import numpy as np
import matplotlib.pyplot as plt

Define as chaves de acesso, token e região do AWS

In [ ]:
AWS_ACCESS_KEY_ID="XXX"
AWS_SECRET_ACCESS_KEY="YYY"
AWS_SESSION_TOKEN="ZZZ"
REGION_NAME="us-east-1"

Cria o cliente S3

In [9]:
s3 = boto3.client('s3',
    aws_access_key_id=AWS_ACCESS_KEY_ID,
    aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
    aws_session_token=AWS_SESSION_TOKEN,
    region_name=REGION_NAME
)

Define o nome do bucket e o nome do arquivo

In [10]:
bucket_name = 'programa-de-bolsas-fernando'
object_key = 'etapa1/ca.csv'

Faz a conexão com o bucket e o objeto do S3 definidos anteriormente

In [11]:
response = s3.get_object(Bucket=bucket_name, Key=object_key)

Faz a leitura do arquivo csv "csv.ca"

In [12]:
df = pd.read_csv(io.BytesIO(response['Body'].read()), encoding="ISO-8859-1", sep=';')

LIMPEZA/TRATAMENTO DE DADOS

Converte a coluna 'data_consulta' para datetime

In [13]:
df['data_consulta'] = pd.to_datetime(df['data_consulta'], dayfirst=True, errors='coerce')

Cria duas novas colunas: 'data' e 'hora'

In [14]:
df['data'] = df['data_consulta'].dt.date
df['hora'] = df['data_consulta'].dt.time

Remove a coluna 'data_consulta'

In [15]:
df = df.drop(columns='data_consulta')

Substitui celulas com espaços em branco para nulo

In [16]:
df = df.replace('', pd.NA) 

Remove as celulas nulas

In [17]:
df = df.dropna()

Converte a coluna 'idade' para int

In [18]:
df['idade'] = df['idade'].astype(int)

ANÁLISE 1 - Total de consultas de todo o período por especialidade para crianças e idosos do sexo feminino de todos os municípios da região Nordeste

In [19]:
filtro = (df['sexo'] == 'F') & ((df['idade'] <= 12) | (df['idade'] >= 60))
analise1 = df[filtro].groupby('especialidade').size().reset_index(name='total_consultas')
analise1 = analise1.sort_values(by='total_consultas', ascending=False)
analise1_top10 = analise1.head(10)

Cria o gráfico de barras das 10 especialidades mais requisitadas por crianças e idosos do sexo feminino de todos os municípios da região Nordeste e salva como png

In [20]:
plt.figure(figsize=(12, 6))
bars = plt.barh(analise1_top10['especialidade'], analise1_top10['total_consultas'], color='skyblue')
for bar in bars:
    plt.text(bar.get_width() + 1, bar.get_y() + bar.get_height() / 2,
             f'{int(bar.get_width())}', va='center')
plt.ylabel('ESPECIALIDADE')
plt.xlabel('TOTAL DE CONSULTAS')
plt.title('Top 10 especialidades mais procuradas por crianças e idosos do sexo feminino')
plt.tight_layout()
plt.gca().invert_yaxis()
plt.savefig('analise1.png')
plt.close()

Salva o dataframe resultante da análise 1 como CSV

In [21]:
analise1.to_csv("analise1.csv", index=False)

Faz upload do dataframe resultante da análise 1 para o bucket

In [22]:
arquivo_local = 'analise1.csv'     
objeto_s3 = 'etapa2/analise1.csv'     
s3.upload_file(
    Filename=arquivo_local,
    Bucket=bucket_name,
    Key=objeto_s3
    )

ANÁLISE 2 - Conversão da idade para float e classificação por faixa etária de homens e mulheres de todo o período de todos os municípios da região Nordeste

In [23]:
df['faixa'] = np.where(df['idade'] >= 18, 'maior de idade', 'menor de idade')
df['idade_float'] = df['idade'].astype(float)
analise2 = df[['idade', 'idade_float', 'faixa']]
analise2_grafico = df['faixa'].value_counts()

Cria o gráfico de pizza da classificação por faixa etaria de homens e mulheres de todo o período de todos os municípios da região Nordeste e salva como png

In [24]:
plt.figure(figsize=(6, 6))
plt.pie(analise2_grafico, labels=analise2_grafico.index, autopct='%1.1f%%', startangle=140, colors=['#66b3ff', '#ff9999'])
plt.title('Classificação por faixa etária das pessoas')
plt.tight_layout()
plt.savefig('analise2.png')
plt.close()

Salva o dataframe resultante da análise 2 como CSV

In [25]:
analise2.to_csv("analise2.csv", index=False)

Faz upload do dataframe resultante da análise 2 para o bucket

In [26]:
arquivo_local = 'analise2.csv'  
objeto_s3 = 'etapa2/analise2.csv'     
s3.upload_file(
    Filename=arquivo_local,
    Bucket=bucket_name,
    Key=objeto_s3
    )

ANÁLISE 3 - Consultas de março com especialidade contendo 'urologia' para homens e mulheres de todos os municípios da região Nordeste

In [27]:
df['data'] = pd.to_datetime(df['data'])
df_marco_urologia = df[
    (df['data'].dt.month == 3) &
    (df['especialidade'].str.strip() == 'UROLOGIA')
].reset_index()
analise3 = df_marco_urologia[['data', 'especialidade']]
analise3_grafico = analise3['data'].dt.date.value_counts().sort_index()

Cria o gráfico de linha da classificação das consultas em março na especialidade 'urologia' de homens e mulheres de todos os municípios da região Nordeste e salva como png

In [28]:
plt.figure(figsize=(10, 5))

datas = pd.to_datetime(analise3_grafico.index).strftime('%d/%m')
intervalo = 3
posicoes_tick = range(0, len(datas), intervalo)
datas_tick = [datas[i] for i in posicoes_tick]

plt.plot(datas, analise3_grafico.values, marker='o', linestyle='-', color='green')
plt.xticks(ticks=posicoes_tick, labels=datas_tick)
plt.xlabel('DATA DA CONSULTA')
plt.ylabel('TOTAL DE CONSULTAS')
plt.title('Consultas em Março na Especialidade "Urologia"')
plt.tight_layout()
plt.grid()
plt.savefig('analise3.png')
plt.close()

Salva o dataframe resultante da análise 3 como CSV

In [29]:
analise3.to_csv("analise3.csv", index=False)

Faz upload do dataframe resultante da análise 3 para o bucket

In [30]:
arquivo_local = 'analise3.csv'     
objeto_s3 = 'etapa2/analise3.csv'     
s3.upload_file(
    Filename=arquivo_local,
    Bucket=bucket_name,
    Key=objeto_s3
    )

Imprime os resultados das 3 análises

In [31]:
print("\nAnálise 1 – Total de consultas de todo o período por especialidade para crianças e idosos do sexo feminino de todos os municípios da região Nordeste:\n")
print(analise1)

print("\nAnálise 2 – Conversão da idade para float e classificação por faixa etária de homens e mulheres de todo o período de todos os municípios da região Nordeste:\n")
print(analise2)

print("\nAnálise 3 – Consultas de março com especialidade contendo 'urologia' para homens e mulheres de todos os municípios da região Nordeste:\n")
print(analise3)


Análise 1 – Total de consultas de todo o período por especialidade para crianças e idosos do sexo feminino de todos os municípios da região Nordeste:

                especialidade  total_consultas
42               OFTALMOLOGIA             1080
43          ONCOLOGIA CLÍNICA              775
20               FISIOTERAPIA              720
2                 CARDIOLOGIA              627
48                  PEDIATRIA              540
..                        ...              ...
61             SERVIÇO SOCIAL                5
5           CIRURGIA CARDIACA                3
52            PRE-TRANSPLANTE                3
41     ODONTOLOGIA PEDIATRICA                2
30  MEDICINA INTERVENCIONISTA                2

[66 rows x 2 columns]

Análise 2 – Conversão da idade para float e classificação por faixa etária de homens e mulheres de todo o período de todos os municípios da região Nordeste:

       idade  idade_float           faixa
0         64         64.0  maior de idade
1         61      